In [ ]:
# Standard code for starting Spark
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Standard Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
 # Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://<bucket name>.s3.amazonaws.com/[INSERT FILE NAME].csv"
spark.sparkContext.addFile(url)
[FILE NAME]_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
[FILE NAME]df.show()

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
dataframe = pd.DataFrame([FILE NAME]df)

In [ ]:
# Does the database merge the track and artist tables?
# If not, load in and merge artists

In [ ]:
# Import dependencies for Machine Learning
 import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import fcluster, linkage, dendrogram
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
# Exploratory Data to understand how we need to preprocess

In [ ]:
# Preprocessing

In [ ]:
# Use Hierarchical clustering
# Read data from DB using PySpark
# Load in Track.csv from S3 into a DataFrame

# Normalize data
from sklearn.preprocessing import normalize
data_scaled = normalize(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)
data_scaled.head()

In [ ]:

# Create hierarchal clustering array using n_clusters determined from dendrogram analysis
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=X, affinity='euclidean', linkage='ward')  
cluster.fit_predict(data_scaled)

In [ ]:
# Plot scatter plot of clusters

In [ ]:
# Trying the next machine learning model
# Which one shall I pick? 